# 1

Scalar()範例：在訓練過程中將數據寫入

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras import datasets, losses, layers, optimizers, metrics, Sequential 

# 定義前處理函數
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y

# 載入 fashion_mnist 資料 
(x, y), (x_test, y_test) = datasets.fashion_mnist.load_data()

train_data_norm, train_label = preprocess(x,y)
test_data_norm, test_label = preprocess(x_test,y_test)

# 建立模型
model = tf.keras.models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
])

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
loss_func = losses.SparseCategoricalCrossentropy()
optimizer = optimizers.Adam()

# 定義訓練及測試的效能衡量指標(Metrics)
train_loss = metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = metrics.SparseCategoricalAccuracy('test_accuracy')

In [ ]:
# 訓練階段
def train_step(model, optimizer, x_train, y_train):
    # 自動微分
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_func(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 計算訓練的效能衡量指標
    train_loss(loss)
    train_accuracy(y_train, predictions)

    # 測試階段
def test_step(model, x_test, y_test):
    # 預測
    predictions = model(x_test)
    # 計算損失
    loss = loss_func(y_test, predictions)

    # 計算測試的效能衡量指標
    test_loss(loss)
    test_accuracy(y_test, predictions)

In [ ]:
import datetime

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 指定訓練的 log 檔名
train_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\train'
# 指定測試的 log 檔名
test_log_dir = '.\\logs\\gradient_tape\\' + current_time + '\\test'

# 開啟檔案
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [ ]:
# 將訓練/測試資料轉成 Tensorflow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_data_norm, train_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data_norm, test_label))

# 每次從 60000 筆訓練資料隨機抽出 64 筆
train_dataset = train_dataset.shuffle(60000).batch(64)
# 每次從 10000 筆測試資料隨機抽出 64 筆
test_dataset = test_dataset.batch(64)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,train_loss.result(), 
                           train_accuracy.result()*100,
                           test_loss.result(), 
                           test_accuracy.result()*100))

    # 每一個 epoch 清除裡面的數值
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

In [ ]:
# 載入 TensorBoard notebook extension，即可在 jupyter notebook 啟動 Tensorboard
%load_ext tensorboard
# 啟動 Tensorboard
%tensorboard --logdir logs/gradient_tape

# 2

Image()範例

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras import datasets, layers, optimizers, Sequential ,metrics

# 載入 fashion_mnist 資料 
(x, y), (x_test, y_test) = datasets.fashion_mnist.load_data()

# 創建一個路徑, 並利用格式化過的時間當作檔案名稱
log_dir='.\\logs\\Imagelog\\'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# (1)創建一個 summarywriter 紀錄器，將要紀錄的資料儲存到 "./Imagelog" 路徑中
summarywriter = tf.summary.create_file_writer(log_dir)  
images = tf.reshape(x, (-1, 28, 28, 1)) 
# (2) 選擇希望使用的記錄器
with summarywriter.as_default(): 
    tf.summary.image('Training data', images, max_outputs=5, step=0)

# 3

利用內建的 Keras Callback 啟動 TensorBoard

In [ ]:
import tensorflow as tf
import os
import datetime

(train_image,train_label),(test_image,test_label)=\
    tf.keras.datasets.fashion_mnist.load_data()

# 對資料集做一個前置處理, 將資料正規到 0~1 之間
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

# 建立模型
def build_model():
    model = tf.keras.models.Sequential()       # 線性疊加
    # 改變平坦輸入
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.5))   # 使用 Dropout 層
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
    return model

model = build_model()

# 編譯模型
model.compile(optimizer= tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = '.\\logs\\tensorboard\\'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# 設定回調函數
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

train_images, train_labels = preprocess(train_image, train_label)
batchsz = 128  # 設定批次大小
# 訓練模型
history = model.fit(train_images, train_labels,epochs= 10,
                    batch_size = batchsz,   # 設定批次訓練大小
                    validation_split=0.2,
                    callbacks=[tensorboard_callback])